In [1]:
#AIT 726-DL1
#Professor: Dr. Lindi Liao
#Project Proposed Solution
#Title: LLMs for Software Engineering -- Code Generation Using LLMs and Evaluation
#Project Team 5
#Mukesh Rajmohan - G01456275
#Utkarsh Jayesh Desai - G01419940
#Praneeth Ravirala - G01448129

In [2]:
#Importing Required Libraries
import torch
from transformers import BertForSequenceClassification, BertTokenizer, AdamW
from torch.utils.data import DataLoader, TensorDataset, RandomSampler
import pandas as pd
from sklearn.model_selection import train_test_split
from collections import defaultdict
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#Importing the dataset
data=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sql_code_dataset.csv')

In [5]:
#Defining Function to Identify SQL Components in the Target Query
def identify_sql_components(answer):
    components = []
    if 'SELECT' in answer:
      components.append('SELECT')
    if 'FROM' in answer:
      components.append('FROM')
    if 'WHERE' in answer:
      components.append('WHERE')
    if 'ORDER BY' in answer:
      components.append('ORDER BY')
    if 'GROUP BY' in answer:
      components.append('GROUP BY')
    return components

In [6]:
#Function to Extract 'SELECT' Detail in the Target Query
def extract_select_details(answer):
    if 'SELECT' in answer and 'FROM' in answer:
        return answer.split('FROM')[0].replace('SELECT', '').strip()
    return ''

In [7]:
#Function to Extract 'FROM' Detail in the Target Query
def extract_from_details(answer):
    if 'FROM' in answer:
        parts = answer.split('FROM')[1].split('WHERE')[0] if 'WHERE' in answer else answer.split('FROM')[1]
        return parts.split()[0].strip()
    return ''

In [8]:
#Function to Extract 'WHERE' Detail in the Target Query
def extract_where_details(answer):
    if 'WHERE' in answer:
        return answer.split('WHERE')[1].split('ORDER BY')[0] if 'ORDER BY' in answer else answer.split('WHERE')[1]
    return ''

In [9]:
#Function to Extract 'ORDER BY' Detail in the Target Query
def extract_order_by_details(answer):
    if 'ORDER BY' in answer:
        return answer.split('ORDER BY')[1].split('GROUP BY')[0].strip() if 'GROUP BY' in answer else answer.split('ORDER BY')[1].strip()
    return ''


In [10]:
#Function to Extract 'GROUP BY' Detail in the Target Query
def extract_group_by_details(answer):
    if 'GROUP BY' in answer:
        return answer.split('GROUP BY')[1].strip()
    return ''

In [11]:
#Applying all the functions to our dataset and retrieving as features
def preprocess_data(data):
    data['sql_components'] = data['answer'].apply(identify_sql_components)
    data['select_columns'] = data['answer'].apply(extract_select_details)
    data['from_tables'] = data['answer'].apply(extract_from_details)
    data['where_conditions'] = data['answer'].apply(extract_where_details)
    data['order_by_columns'] = data['answer'].apply(extract_order_by_details)
    data['group_by_columns'] = data['answer'].apply(extract_group_by_details)
    return data
preprocess_data(data)

,question,context,answer,sql_components,select_columns,from_tables,where_conditions,order_by_columns,group_by_columns
0,How many heads of the departments are older th...,CREATE TABLE head (age INTEGER),SELECT COUNT(*) FROM head WHERE age > 56,"[SELECT, FROM, WHERE]",COUNT(*),head,age > 56,,
1,"List the name, born state and age of the heads...","CREATE TABLE head (name VARCHAR, born_state VA...","SELECT name, born_state, age FROM head ORDER B...","[SELECT, FROM, ORDER BY]","name, born_state, age",head,,age,
2,"List the creation year, name and budget of eac...","CREATE TABLE department (creation VARCHAR, nam...","SELECT creation, name, budget_in_billions FROM...","[SELECT, FROM]","creation, name, budget_in_billions",department,,,
3,What are the maximum and minimum budget of the...,CREATE TABLE department (budget_in_billions IN...,"SELECT MAX(budget_in_billions), MIN(budget_in_...","[SELECT, FROM]","MAX(budget_in_billions), MIN(budget_in_billions)",department,,,
4,What is the average number of employees of the...,CREATE TABLE department (num_employees INTEGER...,SELECT AVG(num_employees) FROM department WHER...,"[SELECT, FROM, WHERE]",AVG(num_employees),department,ranking BETWEEN 10 AND 15,,
...,...,...,...,...,...,...,...,...,...
78572,What time was the match played with a score of...,"CREATE TABLE table_name_35 (time VARCHAR, scor...","SELECT time FROM table_name_35 WHERE score = ""...","[SELECT, FROM, WHERE]",time,table_name_35,"score = ""3-2""",,
78573,On which ground did the team play Aston Villa?,"CREATE TABLE table_name_83 (ground VARCHAR, op...",SELECT ground FROM table_name_83 WHERE opponen...,"[SELECT, FROM, WHERE]",ground,table_name_83,"opponent = ""aston villa""",,
78574,What kind of competition was it at San Siro at...,CREATE TABLE table_name_60 (competition VARCHA...,SELECT competition FROM table_name_60 WHERE gr...,"[SELECT, FROM, WHERE]",competition,table_name_60,"ground = ""san siro"" AND time = ""18:30 gmt""",,
78575,What is the total number of decile for the red...,"CREATE TABLE table_name_34 (decile VARCHAR, na...",SELECT COUNT(decile) FROM table_name_34 WHERE ...,"[SELECT, FROM, WHERE]",COUNT(decile),table_name_34,"name = ""redwood school""",,


In [12]:
#Top Rows of the data after Extracting SQL components and their corresponding details
data.head()

,question,context,answer,sql_components,select_columns,from_tables,where_conditions,order_by_columns,group_by_columns
0,How many heads of the departments are older th...,CREATE TABLE head (age INTEGER),SELECT COUNT(*) FROM head WHERE age > 56,"[SELECT, FROM, WHERE]",COUNT(*),head,age > 56,,
1,"List the name, born state and age of the heads...","CREATE TABLE head (name VARCHAR, born_state VA...","SELECT name, born_state, age FROM head ORDER B...","[SELECT, FROM, ORDER BY]","name, born_state, age",head,,age,
2,"List the creation year, name and budget of eac...","CREATE TABLE department (creation VARCHAR, nam...","SELECT creation, name, budget_in_billions FROM...","[SELECT, FROM]","creation, name, budget_in_billions",department,,,
3,What are the maximum and minimum budget of the...,CREATE TABLE department (budget_in_billions IN...,"SELECT MAX(budget_in_billions), MIN(budget_in_...","[SELECT, FROM]","MAX(budget_in_billions), MIN(budget_in_billions)",department,,,
4,What is the average number of employees of the...,CREATE TABLE department (num_employees INTEGER...,SELECT AVG(num_employees) FROM department WHER...,"[SELECT, FROM, WHERE]",AVG(num_employees),department,ranking BETWEEN 10 AND 15,,


In [14]:
#Joining Components as labels for training
labels = [' | '.join(components) for components in data['sql_components']]

In [15]:
#Displaying top rows of updated dataset
data

,question,context,answer,sql_components,select_columns,from_tables,where_conditions,order_by_columns,group_by_columns
0,How many heads of the departments are older th...,CREATE TABLE head (age INTEGER),SELECT COUNT(*) FROM head WHERE age > 56,"[SELECT, FROM, WHERE]",COUNT(*),head,age > 56,,
1,"List the name, born state and age of the heads...","CREATE TABLE head (name VARCHAR, born_state VA...","SELECT name, born_state, age FROM head ORDER B...","[SELECT, FROM, ORDER BY]","name, born_state, age",head,,age,
2,"List the creation year, name and budget of eac...","CREATE TABLE department (creation VARCHAR, nam...","SELECT creation, name, budget_in_billions FROM...","[SELECT, FROM]","creation, name, budget_in_billions",department,,,
3,What are the maximum and minimum budget of the...,CREATE TABLE department (budget_in_billions IN...,"SELECT MAX(budget_in_billions), MIN(budget_in_...","[SELECT, FROM]","MAX(budget_in_billions), MIN(budget_in_billions)",department,,,
4,What is the average number of employees of the...,CREATE TABLE department (num_employees INTEGER...,SELECT AVG(num_employees) FROM department WHER...,"[SELECT, FROM, WHERE]",AVG(num_employees),department,ranking BETWEEN 10 AND 15,,
...,...,...,...,...,...,...,...,...,...
78572,What time was the match played with a score of...,"CREATE TABLE table_name_35 (time VARCHAR, scor...","SELECT time FROM table_name_35 WHERE score = ""...","[SELECT, FROM, WHERE]",time,table_name_35,"score = ""3-2""",,
78573,On which ground did the team play Aston Villa?,"CREATE TABLE table_name_83 (ground VARCHAR, op...",SELECT ground FROM table_name_83 WHERE opponen...,"[SELECT, FROM, WHERE]",ground,table_name_83,"opponent = ""aston villa""",,
78574,What kind of competition was it at San Siro at...,CREATE TABLE table_name_60 (competition VARCHA...,SELECT competition FROM table_name_60 WHERE gr...,"[SELECT, FROM, WHERE]",competition,table_name_60,"ground = ""san siro"" AND time = ""18:30 gmt""",,
78575,What is the total number of decile for the red...,"CREATE TABLE table_name_34 (decile VARCHAR, na...",SELECT COUNT(decile) FROM table_name_34 WHERE ...,"[SELECT, FROM, WHERE]",COUNT(decile),table_name_34,"name = ""redwood school""",,


In [16]:
data['labels']=labels
data['labels'].head(10)

,labels
0,SELECT | FROM | WHERE
1,SELECT | FROM | ORDER BY
2,SELECT | FROM
3,SELECT | FROM
4,SELECT | FROM | WHERE
5,SELECT | FROM | WHERE
6,SELECT | FROM | WHERE
7,SELECT | FROM | GROUP BY
8,SELECT | FROM | ORDER BY | GROUP BY
9,SELECT | FROM | WHERE


In [17]:
data['labels'].unique()

array(['SELECT | FROM | WHERE', 'SELECT | FROM | ORDER BY',
       'SELECT | FROM', 'SELECT | FROM | GROUP BY',
       'SELECT | FROM | ORDER BY | GROUP BY',
       'SELECT | FROM | WHERE | ORDER BY',
       'SELECT | FROM | WHERE | GROUP BY',
       'SELECT | FROM | WHERE | ORDER BY | GROUP BY'], dtype=object)

In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78577 entries, 0 to 78576
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   question          78577 non-null  object
 1   context           78577 non-null  object
 2   answer            78577 non-null  object
 3   sql_components    78577 non-null  object
 4   select_columns    78577 non-null  object
 5   from_tables       78577 non-null  object
 6   where_conditions  78577 non-null  object
 7   order_by_columns  78577 non-null  object
 8   group_by_columns  78577 non-null  object
 9   labels            78577 non-null  object
dtypes: object(10)
memory usage: 6.0+ MB


In [19]:
#Taking Subset of dataset containing 5000 samples
data=data.sample(5000)

In [ ]:
#Splitting into Training and Testing
#X contains Question and Context
#Y contains Sql components
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data[['question','context']], data['labels'], test_size=0.2, random_state=42)

In [ ]:
context_train=x_train['context']
context_test=x_test['context']
x_train=x_train['question']
x_test=x_test['question']

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
#Defining T5 Transformer Model and its tokenizer for encoding and decoding operations.
T5Tokenizer = T5Tokenizer.from_pretrained('t5-base')
T5Model = T5ForConditionalGeneration.from_pretrained('t5-base').to(device)
#Tokenising input "question" and "context" pair into tokens and encoding tokens for both training and testing datasets.
x_train_encoded = T5Tokenizer(x_train.tolist(), context_train.tolist(), padding=True, truncation=True, return_tensors='pt', max_length=512)
x_test_encoded = T5Tokenizer(x_test.tolist(), context_test.tolist(), padding=True, truncation=True, return_tensors='pt', max_length=512)
#Similarily tokenising and encoding the train label
y_train_encoded = T5Tokenizer(y_train.tolist(),padding=True, truncation=True, return_tensors='pt', max_length=512)
#Defining Optimizer to update the weights and minimising the loss function
T5_Model_optimizer = torch.optim.AdamW(T5Model.parameters(), lr=2e-5)
#Transform the input and label pair to a tensor dataset supported by pytorch
Train_Data_Transform = TensorDataset(x_train_encoded['input_ids'], x_train_encoded['attention_mask'],y_train_encoded['input_ids'])
#Randomly Sample the dataset to batches for parallel processing based on given batch size
Train_Data_Sampler = RandomSampler(Train_Data_Transform)
Train_Data_Loader = DataLoader(Train_Data_Transform, sampler=Train_Data_Sampler, batch_size=8)
# Defining the training function which does the following:
# Set Epochs to 2
#Iterating through batches
#For each batch computing weights based on input_id, attention_mask and label of that corresponding batch
#computing loss function for the corresponding batch
#computing gradient and updating weights by backpropogating to minimize overall loss function.
T5Model.train()
for i in range(2):
    epoch_loss = 0
    for batch in Train_Data_Loader:
        input_id_train=batch[0].to(device)
        attention_mask_train=batch[1].to(device)
        label_train = batch[2].to(device)
        model_outputs = T5Model(input_id_train, attention_mask=attention_mask_train, labels=label_train)
        batch_loss = model_outputs.loss
        epoch_loss = epoch_loss+batch_loss.item()
        batch_loss.backward()
        T5_Model_optimizer.step()
        T5Model.zero_grad()
        torch.cuda.empty_cache()
    Average_Epoch_Loss = epoch_loss / len(Train_Data_Loader)
    print('Epoch '+str(i)+' Loss = '+str(Average_Epoch_Loss))

Epoch 0 Loss = 0.5315563783270773
Epoch 1 Loss = 0.020968087995075622


In [ ]:
#Taking Subset of Test Sentences
x_test_encoded1=x_test_encoded[:500]
T5Model.eval()
with torch.no_grad():
    all_outputs = []
    for i in range(500):
        generated_output = T5Model.generate(
            input_ids=x_test_encoded1['input_ids'][i:i+1].to(device),
            attention_mask=x_test_encoded1['attention_mask'][i:i+1].to(device),
            max_length=100,
            num_beams=5,
            early_stopping=True
        )
        all_outputs.append(generated_output)
        torch.cuda.empty_cache()
#Predicting SQl components based on given question and context
#With this Stage 1 SQL Component Identification is done
predicted_code0=[]
for i in all_outputs:
  pred=T5Tokenizer.decode(i[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
  predicted_code0.append(pred)
for input_text, code in zip(y_test, predicted_code0):
    print(f"Input: {input_text}\nGenerated Code:\n{code}\n")

Input: SELECT | FROM | WHERE
Generated Code:
SELECT | FROM | WHERE

Input: SELECT | FROM | WHERE
Generated Code:
SELECT | FROM | WHERE

Input: SELECT | FROM | WHERE
Generated Code:
SELECT | FROM | WHERE

Input: SELECT | FROM | WHERE
Generated Code:
SELECT | FROM | WHERE

Input: SELECT | FROM | WHERE
Generated Code:
SELECT | FROM | WHERE

Input: SELECT | FROM | WHERE
Generated Code:
SELECT | FROM | WHERE

Input: SELECT | FROM | WHERE
Generated Code:
SELECT | FROM | WHERE

Input: SELECT | FROM | WHERE
Generated Code:
SELECT | FROM | WHERE

Input: SELECT | FROM | WHERE
Generated Code:
SELECT | FROM | WHERE

Input: SELECT | FROM | WHERE
Generated Code:
SELECT | FROM | WHERE

Input: SELECT | FROM | WHERE
Generated Code:
SELECT | FROM | WHERE

Input: SELECT | FROM | WHERE
Generated Code:
SELECT | FROM | WHERE

Input: SELECT | FROM | WHERE
Generated Code:
SELECT | FROM | WHERE

Input: SELECT | FROM | WHERE
Generated Code:
SELECT | FROM | WHERE

Input: SELECT | FROM | WHERE
Generated Code:
SEL

In [20]:
#Stage 2:
#Dividing the Filling SQL Query process into subtasks
subtasks = ["select_columns", "from_tables", "where_conditions","order_by_columns","group_by_columns"]

#Filling Details of each subtask(sql component) into respective lists

select_data = []
from_data = []
where_data = []
order_by_data = []
group_by_data = []

#Iterating through dataset and retriving corresponding lists containing fill component, question and detail of the component.
for subtask in subtasks:
    for _, row in data.iterrows():
        input_text = f"Fill {subtask}: {row['question']} | {row['context']} | {row['sql_components']}"
        target_text = row[subtask] if not pd.isna(row[subtask]) else "NULL"
        entry = {"input": input_text, "label": target_text}

        if subtask == "select_columns":
            select_data.append(entry)
        elif subtask == "from_tables":
            from_data.append(entry)
        elif subtask == "where_conditions":
            where_data.append(entry)
        elif subtask == "order_by_columns":
            order_by_data.append(entry)
        elif subtask == "group_by_columns":
            group_by_data.append(entry)

#Converting Subtask lists into respective datasets for training and testing seperately
select_df = pd.DataFrame(select_data)
from_df = pd.DataFrame(from_data)
where_df = pd.DataFrame(where_data)
order_by_df = pd.DataFrame(order_by_data)
group_by_df = pd.DataFrame(group_by_data)

In [21]:
data1=select_df
data2=from_df
data3=where_df
data4=order_by_df
data5=group_by_df

print(data1.head(1))
print(data2.head(1))
print(data3.head(1))
print(data4.head(1))
print(data5.head(1))

                                               input   label
0  Fill select_columns: Who is the rookie goalkee...  rookie
                                               input          label
0  Fill from_tables: Who is the rookie goalkeeper...  table_name_70
                                               input  \
0  Fill where_conditions: Who is the rookie goalk...   

                        label  
0   goalkeeper = "mike gabel"  
                                               input label
0  Fill order_by_columns: Who is the rookie goalk...      
                                               input label
0  Fill group_by_columns: Who is the rookie goalk...      


In [ ]:
#Splitting 'SELECT' dataset into train and test
from sklearn.model_selection import train_test_split
x1_train, x1_test, y1_train, y1_test = train_test_split(data1['input'],
                                                    data1['label'], test_size=0.2, random_state=42)

In [ ]:
#Splitting 'FROM' dataset into train and test
from sklearn.model_selection import train_test_split
x2_train, x2_test, y2_train, y2_test = train_test_split(data2['input'],
                                                    data2['label'], test_size=0.2, random_state=42)

In [ ]:
#Splitting 'WHERE' dataset into train and test
from sklearn.model_selection import train_test_split
x3_train, x3_test, y3_train, y3_test = train_test_split(data3['input'],
                                                    data3['label'], test_size=0.2, random_state=42)

In [ ]:
#Splitting 'ORDER BY' dataset into train and test
from sklearn.model_selection import train_test_split
x4_train, x4_test, y4_train, y4_test = train_test_split(data4['input'],
                                                    data4['label'], test_size=0.2, random_state=42)

In [ ]:
#Splitting 'GROUP BY' dataset into train and test
from sklearn.model_selection import train_test_split
x5_train, x5_test, y5_train, y5_test = train_test_split(data5['input'],
                                                    data5['label'], test_size=0.2, random_state=42)

In [ ]:
#Encoding 'Train and Test' of 'SELECT' dataset using T5 Tokenizer
x1_train_encoded = T5Tokenizer(x1_train.tolist(), padding=True, truncation=True,
                              return_tensors='pt', max_length=512)
x1_test_encoded = T5Tokenizer(x1_test.tolist(), padding=True, truncation=True,
                             return_tensors='pt', max_length=512)
y1_train_encoded = T5Tokenizer(y1_train.tolist(),padding=True, truncation=True, return_tensors='pt', max_length=512)

In [ ]:
#Encoding 'Train and Test' of 'FROM' dataset using T5 Tokenizer
x2_train_encoded = T5Tokenizer(x2_train.tolist(), padding=True, truncation=True,
                              return_tensors='pt', max_length=512)
x2_test_encoded = T5Tokenizer(x2_test.tolist(), padding=True, truncation=True,
                             return_tensors='pt', max_length=512)
y2_train_encoded = T5Tokenizer(y2_train.tolist(),padding=True, truncation=True, return_tensors='pt', max_length=512)

In [ ]:
#Encoding 'Train and Test' of 'WHERE' dataset using T5 Tokenizer
x3_train_encoded = T5Tokenizer(x3_train.tolist(), padding=True, truncation=True,
                              return_tensors='pt', max_length=512)
x3_test_encoded = T5Tokenizer(x3_test.tolist(), padding=True, truncation=True,
                             return_tensors='pt', max_length=512)
y3_train_encoded = T5Tokenizer(y3_train.tolist(),padding=True, truncation=True, return_tensors='pt', max_length=512)

In [ ]:
#Encoding 'Train and Test' of 'ORDER BY' dataset using T5 Tokenizer
x4_train_encoded = T5Tokenizer(x4_train.tolist(), padding=True, truncation=True,
                              return_tensors='pt', max_length=512)
x4_test_encoded = T5Tokenizer(x4_test.tolist(), padding=True, truncation=True,
                             return_tensors='pt', max_length=512)
y4_train_encoded = T5Tokenizer(y4_train.tolist(),padding=True, truncation=True, return_tensors='pt', max_length=512)

In [ ]:
#Encoding 'Train and Test' of 'GROUP BY' dataset using T5 Tokenizer
x5_train_encoded = T5Tokenizer(x5_train.tolist(), padding=True, truncation=True,
                              return_tensors='pt', max_length=512)
x5_test_encoded = T5Tokenizer(x5_test.tolist(), padding=True, truncation=True,
                             return_tensors='pt', max_length=512)
y5_train_encoded = T5Tokenizer(y5_train.tolist(),padding=True, truncation=True, return_tensors='pt', max_length=512)

Stage2:

Training and Predicting Select Fill detail

In [ ]:
#Transforming and Defining Model Parameters
Transform_Train_Tensor = TensorDataset(x1_train_encoded['input_ids'], x1_train_encoded['attention_mask'],
                                       y1_train_encoded['input_ids'])
Sample_Train_Tensor = RandomSampler(Transform_Train_Tensor)
Load_Train_Batches = DataLoader(Transform_Train_Tensor, sampler=Sample_Train_Tensor, batch_size=8)
T5ModelOptimizer = torch.optim.AdamW(T5Model.parameters(), lr=2e-5)

In [ ]:
#Training 'SELECT' dataset with T5 with similar functionality as mentioned before.
T5Model.train()
for i in range(2):
    epoch_loss = 0
    for batch in Load_Train_Batches:
        input_id_train=batch[0].to(device)
        attention_mask_train=batch[1].to(device)
        label_train = batch[2].to(device)
        model_outputs = T5Model(input_id_train, attention_mask=attention_mask_train, labels=label_train)
        batch_loss = model_outputs.loss
        epoch_loss = epoch_loss+batch_loss.item()
        batch_loss.backward()
        T5_Model_optimizer.step()
        T5Model.zero_grad()
        torch.cuda.empty_cache()
    Average_Epoch_Loss = epoch_loss / len(Load_Train_Batches)
    print('Epoch '+str(i)+' Loss = '+str(Average_Epoch_Loss))

Epoch 0 Loss = 1.185669342212379
Epoch 1 Loss = 0.14307400258630515


In [ ]:
#Taking Subset of test dataset
x1_test_encoded1=x1_test_encoded[:500]
x1_test_encoded1

{'input_ids': tensor([[12607,  1738,   834,  ...,     0,     0,     0],
         [12607,  1738,   834,  ...,     0,     0,     0],
         [12607,  1738,   834,  ...,     0,     0,     0],
         ...,
         [12607,  1738,   834,  ...,     0,     0,     0],
         [12607,  1738,   834,  ...,     0,     0,     0],
         [12607,  1738,   834,  ...,     0,     0,     0]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]])}

In [ ]:
#Predicting 'SELECT' fill detail
T5Model.eval()
with torch.no_grad():
    all_outputs = []
    for i in range(500):
        generated_output = T5Model.generate(
            input_ids=x1_test_encoded1['input_ids'][i:i+1].to(device),
            attention_mask=x1_test_encoded1['attention_mask'][i:i+1].to(device),
            max_length=100,
            num_beams=5,
            early_stopping=True
        )
        all_outputs.append(generated_output)
        torch.cuda.empty_cache()

In [ ]:
#Displaying Test fill detail and Predicted fill detail for 'SELECT'
predicted_code1=[]
for i in all_outputs:
  pred=T5Tokenizer.decode(i[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
  predicted_code1.append(pred)
for input_text, code in zip(y1_test, predicted_code1):
    print(f"Input: {input_text}\nGenerated Code:\n{code}\n")

Input: club
Generated Code:
club

Input: illustrator
Generated Code:
illustrator

Input: departure_pune
Generated Code:
departure_pune

Input: nfl_team
Generated Code:
nfl_team

Input: player
Generated Code:
player

Input: MIN(stage)
Generated Code:
stage

Input: weight
Generated Code:
weight

Input: chord
Generated Code:
Chord

Input: city_state
Generated Code:
city_state

Input: income_poverty_f
Generated Code:
income_poverty_f

Input: COUNT(total)
Generated Code:
COUNT(total)

Input: bike
Generated Code:
bike

Input: COUNT(round)
Generated Code:
round

Input: MAX(total)
Generated Code:
COUNT(total)

Input: date
Generated Code:
date

Input: SUM(game)
Generated Code:
MIN(game)

Input: team
Generated Code:
team

Input: location
Generated Code:
location

Input: play_by_play
Generated Code:
studio_host

Input: COUNT(turbine_manufacturer)
Generated Code:
COUNT(turbine_manufacturer)

Input: home_town
Generated Code:
home_town

Input: player
Generated Code:
player

Input: venue
Generated Co

In [ ]:
#Stage2: Training and Predicting FROM Fill detail
#Transforming and Defining Model Parameters
Transform_Train_Tensor = TensorDataset(x2_train_encoded['input_ids'], x2_train_encoded['attention_mask'],
                                       y2_train_encoded['input_ids'])
Sample_Train_Tensor = RandomSampler(Transform_Train_Tensor)
Load_Train_Batches = DataLoader(Transform_Train_Tensor, sampler=Sample_Train_Tensor, batch_size=8)
T5ModelOptimizer = torch.optim.AdamW(T5Model.parameters(), lr=2e-5)

In [ ]:
#Training 'FROM' dataset with T5 with similar functionality as mentioned before.
T5Model.train()
for i in range(2):
    epoch_loss = 0
    for batch in Load_Train_Batches:
        input_id_train=batch[0].to(device)
        attention_mask_train=batch[1].to(device)
        label_train = batch[2].to(device)
        model_outputs = T5Model(input_id_train, attention_mask=attention_mask_train, labels=label_train)
        batch_loss = model_outputs.loss
        epoch_loss = epoch_loss+batch_loss.item()
        batch_loss.backward()
        T5_Model_optimizer.step()
        T5Model.zero_grad()
        torch.cuda.empty_cache()
    Average_Epoch_Loss = epoch_loss / len(Load_Train_Batches)
    print('Epoch '+str(i)+' Loss = '+str(Average_Epoch_Loss))

Epoch 0 Loss = 0.03302684563607909
Epoch 1 Loss = 0.011325780261890031


In [ ]:
#Taking Subset of test dataset
x2_test_encoded1=x2_test_encoded[:500]
x2_test_encoded1

{'input_ids': tensor([[12607,    45,   834,  ...,     0,     0,     0],
         [12607,    45,   834,  ...,     0,     0,     0],
         [12607,    45,   834,  ...,     0,     0,     0],
         ...,
         [12607,    45,   834,  ...,     0,     0,     0],
         [12607,    45,   834,  ...,     0,     0,     0],
         [12607,    45,   834,  ...,     0,     0,     0]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]])}

In [ ]:
#Predicting 'FROM' fill detail
T5Model.eval()
with torch.no_grad():
    all_outputs = []
    for i in range(500):
        generated_output = T5Model.generate(
            input_ids=x2_test_encoded1['input_ids'][i:i+1].to(device),
            attention_mask=x2_test_encoded1['attention_mask'][i:i+1].to(device),
            max_length=100,
            num_beams=5,
            early_stopping=True
        )
        all_outputs.append(generated_output)
        torch.cuda.empty_cache()

In [ ]:
#Displaying Test fill detail and Predicted fill detail for 'FROM'
predicted_code2=[]
for i in all_outputs:
  pred=T5Tokenizer.decode(i[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
  predicted_code2.append(pred)

In [ ]:
for input_text, code in zip(y2_test, predicted_code2):
    print(f"Input: {input_text}\nGenerated Code:\n{code}\n")

Input: table_name_96
Generated Code:
table_name_96

Input: table_name_79
Generated Code:
table_name_79

Input: table_29301050_1
Generated Code:
table_29301050_1

Input: table_2508633_6
Generated Code:
table_2508633_6

Input: table_name_24
Generated Code:
table_name_24

Input: table_15088557_1
Generated Code:
table_15088557_1

Input: table_name_41
Generated Code:
table_name_41

Input: table_name_73
Generated Code:
table_name_73

Input: table_name_52
Generated Code:
table_name_52

Input: table_13618358_1
Generated Code:
table_13618358_1

Input: table_name_4
Generated Code:
table_name_4

Input: table_name_7
Generated Code:
table_name_7

Input: table_26920192_5
Generated Code:
table_26920192_5

Input: table_name_74
Generated Code:
table_name_74

Input: table_name_14
Generated Code:
table_name_14

Input: table_name_61
Generated Code:
table_name_61

Input: table_name_21
Generated Code:
table_name_21

Input: table_name_12
Generated Code:
table_name_12

Input: table_name_80
Generated Code:
tab

In [ ]:
#Stage2: Training and Predicting WHERE Fill detail
#Transforming and Defining Model Parameters
Transform_Train_Tensor = TensorDataset(x3_train_encoded['input_ids'], x3_train_encoded['attention_mask'],
                                       y3_train_encoded['input_ids'])
Sample_Train_Tensor = RandomSampler(Transform_Train_Tensor)
Load_Train_Batches = DataLoader(Transform_Train_Tensor, sampler=Sample_Train_Tensor, batch_size=8)
T5ModelOptimizer = torch.optim.AdamW(T5Model.parameters(), lr=2e-5)

In [ ]:
#Training 'WHERE' dataset with T5 with similar functionality as mentioned before.
T5Model.train()
for i in range(2):
    epoch_loss = 0
    for batch in Load_Train_Batches:
        input_id_train=batch[0].to(device)
        attention_mask_train=batch[1].to(device)
        label_train = batch[2].to(device)
        model_outputs = T5Model(input_id_train, attention_mask=attention_mask_train, labels=label_train)
        batch_loss = model_outputs.loss
        epoch_loss = epoch_loss+batch_loss.item()
        batch_loss.backward()
        T5_Model_optimizer.step()
        T5Model.zero_grad()
        torch.cuda.empty_cache()
    Average_Epoch_Loss = epoch_loss / len(Load_Train_Batches)
    print('Epoch '+str(i)+' Loss = '+str(Average_Epoch_Loss))

Epoch 0 Loss = 0.1773597779497504
Epoch 1 Loss = 0.08164384141936898


In [ ]:
#Taking Subset of test dataset
x3_test_encoded1=x3_test_encoded[:500]
x3_test_encoded1

{'input_ids': tensor([[12607,   213,   834,  ...,     0,     0,     0],
         [12607,   213,   834,  ...,     0,     0,     0],
         [12607,   213,   834,  ...,     0,     0,     0],
         ...,
         [12607,   213,   834,  ...,     0,     0,     0],
         [12607,   213,   834,  ...,     0,     0,     0],
         [12607,   213,   834,  ...,     0,     0,     0]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]])}

In [ ]:
#Predicting 'WHERE' fill detail
T5Model.eval()
with torch.no_grad():
    all_outputs = []
    for i in range(500):
        generated_output = T5Model.generate(
            input_ids=x3_test_encoded1['input_ids'][i:i+1].to(device),
            attention_mask=x3_test_encoded1['attention_mask'][i:i+1].to(device),
            max_length=100,
            num_beams=5,
            early_stopping=True
        )
        all_outputs.append(generated_output)
        torch.cuda.empty_cache()

In [ ]:
#Displaying Test fill detail and Predicted fill detail for 'FROM'
predicted_code3=[]
for i in all_outputs:
  pred=T5Tokenizer.decode(i[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
  predicted_code3.append(pred)
for input_text, code in zip(y3_test, predicted_code3):
    print(f"Input: {input_text}\nGenerated Code:\n{code}\n")

Input:  played = "22" AND drawn = "0" AND lost = "14" AND losing_bp = "5"
Generated Code:
played = "22 rounds" AND drawn = "0" AND losing_bp = 5

Input:  year = 1987
Generated Code:
year = 1987

Input:  arrival_lonavla = "22:22"
Generated Code:
arrival_lonavla = "22:22"

Input:  pick__number = 151
Generated Code:
pick__number = "151"

Input:  country = "united states" AND money___$__ = 356
Generated Code:
country = "usa" AND money___$__ = "356"

Input:  mountains_classification = "Aitor Osa" AND winner = "Aitor González"
Generated Code:
mountains_classification = "aitor osa" AND winner = "aitor gonzález"

Input:  denomination = "three paise"
Generated Code:
denomination = "three paise"

Input:  minor_seventh = "f"
Generated Code:
minor_seventh = "f"

Input:  status = "2nd runner-up"
Generated Code:
status = 2nd runner-up

Input:  services_ & _cons_truction_b = "72.5"
Generated Code:
services_ = "72.5" AND _cons_truction_b = "72.5"

Input:  bronze > 1 AND silver = 1
Generated Code:
bron

In [ ]:
#Stage2: Training and Predicting ORDER BY Fill detail
#Transforming and Defining Model Parameters
Transform_Train_Tensor = TensorDataset(x4_train_encoded['input_ids'], x4_train_encoded['attention_mask'],
                                       y4_train_encoded['input_ids'])
Sample_Train_Tensor = RandomSampler(Transform_Train_Tensor)
Load_Train_Batches = DataLoader(Transform_Train_Tensor, sampler=Sample_Train_Tensor, batch_size=8)
T5ModelOptimizer = torch.optim.AdamW(T5Model.parameters(), lr=2e-5)

In [ ]:
#Training 'ORDER BY' dataset with T5 with similar functionality as mentioned before.
T5Model.train()
for i in range(2):
    epoch_loss = 0
    for batch in Load_Train_Batches:
        input_id_train=batch[0].to(device)
        attention_mask_train=batch[1].to(device)
        label_train = batch[2].to(device)
        model_outputs = T5Model(input_id_train, attention_mask=attention_mask_train, labels=label_train)
        batch_loss = model_outputs.loss
        epoch_loss = epoch_loss+batch_loss.item()
        batch_loss.backward()
        T5_Model_optimizer.step()
        T5Model.zero_grad()
        torch.cuda.empty_cache()
    Average_Epoch_Loss = epoch_loss / len(Load_Train_Batches)
    print('Epoch '+str(i)+' Loss = '+str(Average_Epoch_Loss))

Epoch 0 Loss = 0.04904623072477989
Epoch 1 Loss = 0.01211609827639768


In [ ]:
#Taking Subset of test dataset
x4_test_encoded1=x4_test_encoded[:500]
x4_test_encoded1

{'input_ids': tensor([[12607,   455,   834,  ...,     0,     0,     0],
         [12607,   455,   834,  ...,     0,     0,     0],
         [12607,   455,   834,  ...,     0,     0,     0],
         ...,
         [12607,   455,   834,  ...,     0,     0,     0],
         [12607,   455,   834,  ...,     0,     0,     0],
         [12607,   455,   834,  ...,     0,     0,     0]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]])}

In [ ]:
#Predicting 'ORDER BY' fill detail
T5Model.eval()
with torch.no_grad():
    all_outputs = []
    for i in range(500):
        generated_output = T5Model.generate(
            input_ids=x4_test_encoded1['input_ids'][i:i+1].to(device),
            attention_mask=x4_test_encoded1['attention_mask'][i:i+1].to(device),
            max_length=100,
            num_beams=5,
            early_stopping=True
        )
        all_outputs.append(generated_output)
        torch.cuda.empty_cache()

In [ ]:
#Displaying Test fill detail and Predicted fill detail for 'ORDER BY'
predicted_code4=[]
for i in all_outputs:
  pred=T5Tokenizer.decode(i[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
  predicted_code4.append(pred)
for input_text, code in zip(y4_test, predicted_code4):
    print(f"Input: {input_text}\nGenerated Code:\n{code}\n")

Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: open_date DESC LIMIT 3
Generate

In [ ]:
#Stage2: Training and Predicting GROUP BY Fill detail
#Transforming and Defining Model Parameters
Transform_Train_Tensor = TensorDataset(x5_train_encoded['input_ids'], x5_train_encoded['attention_mask'],
                                       y5_train_encoded['input_ids'])
Sample_Train_Tensor = RandomSampler(Transform_Train_Tensor)
Load_Train_Batches = DataLoader(Transform_Train_Tensor, sampler=Sample_Train_Tensor, batch_size=8)
T5ModelOptimizer = torch.optim.AdamW(T5Model.parameters(), lr=2e-5)

In [ ]:
#Training 'GROUP BY' dataset with T5 with similar functionality as mentioned before.
T5Model.train()
for i in range(2):
    epoch_loss = 0
    for batch in Load_Train_Batches:
        input_id_train=batch[0].to(device)
        attention_mask_train=batch[1].to(device)
        label_train = batch[2].to(device)
        model_outputs = T5Model(input_id_train, attention_mask=attention_mask_train, labels=label_train)
        batch_loss = model_outputs.loss
        epoch_loss = epoch_loss+batch_loss.item()
        batch_loss.backward()
        T5_Model_optimizer.step()
        T5Model.zero_grad()
        torch.cuda.empty_cache()
    Average_Epoch_Loss = epoch_loss / len(Load_Train_Batches)
    print('Epoch '+str(i)+' Loss = '+str(Average_Epoch_Loss))

Epoch 0 Loss = 0.00866663793366024
Epoch 1 Loss = 0.006246779331755534


In [ ]:
#Taking Subset of test dataset
x5_test_encoded1=x5_test_encoded[:500]
x5_test_encoded1

{'input_ids': tensor([[12607,   563,   834,  ...,     0,     0,     0],
         [12607,   563,   834,  ...,     0,     0,     0],
         [12607,   563,   834,  ...,     0,     0,     0],
         ...,
         [12607,   563,   834,  ...,     0,     0,     0],
         [12607,   563,   834,  ...,     0,     0,     0],
         [12607,   563,   834,  ...,     0,     0,     0]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]])}

In [ ]:
#Predicting 'GROUP BY' fill detail
T5Model.eval()
with torch.no_grad():
    all_outputs = []
    for i in range(500):
        generated_output = T5Model.generate(
            input_ids=x5_test_encoded1['input_ids'][i:i+1].to(device),
            attention_mask=x5_test_encoded1['attention_mask'][i:i+1].to(device),
            max_length=100,
            num_beams=5,
            early_stopping=True
        )
        all_outputs.append(generated_output)
        torch.cuda.empty_cache()

In [ ]:
#Displaying Test fill detail and Predicted fill detail for 'GROUP BY'
#Stage 2 is done
predicted_code5=[]
for i in all_outputs:
  pred=T5Tokenizer.decode(i[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
  predicted_code5.append(pred)
for input_text, code in zip(y5_test, predicted_code5):
    print(f"Input: {input_text}\nGenerated Code:\n{code}\n")

Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Generated Code:


Input: 
Gene

In [ ]:
#Stage 3: Query Reconstruction
#Aggregating all the above test components and predicted components
y1=y1_test[:500].reset_index(drop=True)
y2=y2_test[:500].reset_index(drop=True)
y3=y3_test[:500].reset_index(drop=True)
y4=y4_test[:500].reset_index(drop=True)
y5=y5_test[:500].reset_index(drop=True)

In [ ]:
y0=y_test[:500].reset_index(drop=True)

In [ ]:
=x_test[:500].tolist()

In [ ]:
#Through For Loop Displaying Test Code and Predicted code by:
# Concatenating Test Components and Test Fill details
#Concatenating Predicted components and Predicted Fill details
test_list=[]
predicted_list=[]
for i in range(500):
  print('Test Code: ')
  y00=y0[i].split('|')
  y00 = [k.strip() for k in y00]
  t=''
  if 'SELECT' in y00:
    t=t+'SELECT '+y1[i]
  if 'FROM' in y00:
    t=t+' FROM '+y2[i]
  if 'WHERE' in y00:
    t=t+' WHERE '+y3[i]
  if 'ORDER BY' in y00:
    t=t+' ORDER BY '+y4[i]
  if 'GROUP BY' in y00:
    t=t+' GROUP BY '+y5[i]
  print(t)
  test_list.append(t)
  print('Predicted Code: ')
  predicted00=predicted_code0[i].split('|')
  predicted00 = [k.strip() for k in predicted00]
  p=''
  if 'SELECT' in predicted00:
    p=p+predicted00[0]+' '+predicted_code1[i]
  if 'FROM' in predicted00:
    p=p+' '+predicted00[1]+' '+predicted_code2[i]
  if 'WHERE' in predicted00:
    p=p+' '+predicted00[2]+' '+predicted_code3[i]
  if 'ORDER BY' in predicted00:
    p=p+' '+predicted00[3]+' '+predicted_code4[i]
  if 'GROUP BY' in predicted00:
    p=p+' '+predicted00[4]+' '+predicted_code5[i]
  print(p)
  predicted_list.append(p)

Test Code: 
SELECT club FROM table_name_96 WHERE  played = "22" AND drawn = "0" AND lost = "14" AND losing_bp = "5"
Predicted Code: 
SELECT club FROM table_name_96 WHERE played = "22 rounds" AND drawn = "0" AND losing_bp = 5
Test Code: 
SELECT illustrator FROM table_name_79 WHERE  year = 1987
Predicted Code: 
SELECT illustrator FROM table_name_79 WHERE year = 1987
Test Code: 
SELECT departure_pune FROM table_29301050_1 WHERE  arrival_lonavla = "22:22"
Predicted Code: 
SELECT departure_pune FROM table_29301050_1 WHERE arrival_lonavla = "22:22"
Test Code: 
SELECT nfl_team FROM table_2508633_6 WHERE  pick__number = 151
Predicted Code: 
SELECT nfl_team FROM table_2508633_6 WHERE pick__number = "151"
Test Code: 
SELECT player FROM table_name_24 WHERE  country = "united states" AND money___$__ = 356
Predicted Code: 
SELECT player FROM table_name_24 WHERE country = "usa" AND money___$__ = "356"
Test Code: 
SELECT MIN(stage) FROM table_15088557_1 WHERE  mountains_classification = "Aitor Osa" A

In [ ]:
pip install rapidfuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 42.4 MB/s eta 0:00:00


In [ ]:
#Using Test List and Predicted list retrieving Fuzz Ratio Metric Score
from rapidfuzz import fuzz
total=0
for i in range(0,500):
  result=fuzz.ratio(test_list[i],predicted_list[i])
  total=total+result

print(total/500)

91.4104434809133


In [ ]:
pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=6c5b0d52babbcfb6b058c87fa7ab96f89eaeacbfe52d1c40a4761ce864e9066e
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
#Using Test List and Predicted list retrieving Rouge(R1,R2,RL) Metric Score
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=False)
R1=[]
R2=[]
RL=[]
for i in range(0,500):
  scores = scorer.score(test_list[i], predicted_list[i])
  R1.append(scores['rouge1'].fmeasure)
  R2.append(scores['rouge2'].fmeasure)
  RL.append(scores['rougeL'].fmeasure)

for i in range(0,500):
  sum_R1=sum(R1)
  sum_R2=sum(R2)
  sum_RL=sum(RL)

print('Rouge1 Score: '+str(sum_R1/500))
print('Rouge2 Score: '+str(sum_R2/500))
print('RougeL Score: '+str(sum_RL/500))

Rouge1 Score: 0.9254991466021099
Rouge2 Score: 0.8604519015826005
RougeL Score: 0.9152792638922922


In [ ]:
pip install nltk

In [ ]:
#Using Test List and Predicted list retrieving BLEU Metric Score
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
bleu=[]
for i in range(0,500):
  reference_tokens = [test_list[i].split()]
  generated_tokens = predicted_list[i].split()
  smoothing = SmoothingFunction().method2
  bleu_score = sentence_bleu(reference_tokens, generated_tokens, smoothing_function=smoothing)
  bleu.append(bleu_score)

sum_bleu=(sum(bleu))/500
print(f"BLEU Score: {sum_bleu:.4f}")


BLEU Score: 0.7165


In [ ]:
#References:
#[1].T5. (n.d.). https://huggingface.co/docs/transformers/en/model_doc/t5
#[2]. Dutta, N. (2024, October 7). Fuzzy String Matching – a hands-on guide. Analytics Vidhya. https://www.analyticsvidhya.com/blog/2021/07/fuzzy-string-matching-a-hands-on-guide/
#[3].Isaacndirangumuturi. (2023, October 5). Code Generation with T5 Transformer. Kaggle. https://www.kaggle.com/code/isaacndirangumuturi/code-generation-with-t5-transformer
#[4].b-mc2/sql-create-context · Datasets at Hugging Face. (2001, May 12). https://huggingface.co/datasets/b-mc2/sql-create-context?row=0
#[5]. [PDF] Seq2SQL: Generating Structured Queries from Natural Language using Reinforcement Learning | Semantic Scholar. (2017, August 31). https://www.semanticscholar.org/reader/cbd569036fc72ae7ff747350b91816440282596b